In [1]:

import ase
import numpy as np
from ase.visualize import view
from ase.io import read, write
from ase.io import cif, vasp, lammpsdata
from ase.build import make_supercell, sort, add_adsorbate, add_vacuum
import nglview

In [2]:
atoms = vasp.read_vasp('orthorhombic.POSCAR.vasp')
len(atoms)

60

In [3]:
v = view(atoms, viewer='ngl')
v

In [4]:
atoms.cell

Cell([4.8050279617, 4.8050279617, 13.1162538528])

In [5]:
a = atoms.cell[0][0]
a

4.8050279617

In [6]:
b = atoms.cell[1][1]
b

4.8050279617

In [7]:
c = atoms.cell[2][2]
c

13.1162538528

In [8]:
atoms.positions

array([[ 0.        ,  0.        ,  1.93994633],
       [ 0.        ,  0.        , 11.17630772],
       [ 1.60167589,  3.20335207,  0.24609594],
       [ 3.20335207,  1.60167589, 12.8701582 ],
       [ 1.60167589,  3.20335207,  4.12598832],
       [ 3.20335207,  1.60167589,  8.99026516],
       [ 3.20335207,  1.60167603,  2.43213823],
       [ 1.60167589,  3.20335178, 10.68411562],
       [ 3.20335207,  1.60167603,  6.31203088],
       [ 1.60167589,  3.20335178,  6.80422337],
       [ 0.        ,  0.        ,  4.6181804 ],
       [ 0.        ,  0.        ,  8.49807383],
       [ 0.        ,  0.        ,  8.49807227],
       [ 0.        ,  0.        ,  4.61818158],
       [ 1.60167589,  3.20335207,  6.80422258],
       [ 3.20335207,  1.60167589,  6.31203127],
       [ 1.60167589,  3.20335207, 10.68411562],
       [ 3.20335207,  1.60167589,  2.43213823],
       [ 3.20335207,  1.60167603,  8.99026516],
       [ 1.60167589,  3.20335178,  4.1259887 ],
       [ 3.20335207,  1.60167603, 12.870

In [9]:
unique = np.unique(atoms.positions.round(decimals=3), axis=0)

In [10]:
len(unique)

30

In [11]:
Al = atoms[0:12]

In [12]:
v = view(Al, viewer='ngl')
v

In [13]:
O = atoms[24:24+18]
O

Atoms(symbols='O18', pbc=True, cell=[4.8050279617, 4.8050279617, 13.1162538528])

In [14]:
v = view(O, viewer='ngl')
v

In [15]:
combined = Al + O
combined

Atoms(symbols='Al12O18', pbc=True, cell=[4.8050279617, 4.8050279617, 13.1162538528])

In [16]:
lammpsdata.write_lammps_data("Al2O3_small.data", combined)

In [17]:
cell = combined.cell
cell

Cell([4.8050279617, 4.8050279617, 13.1162538528])

In [18]:
a = cell[0][0]
target_a = 100
scalfac_a = target_a/a
print(scalfac_a)
round(scalfac_a)

20.81153341813653


21

In [19]:
b = cell[1][1]
target_b = 100
scalfac_b = target_b/b
print(scalfac_b)
round(scalfac_b)

20.81153341813653


21

In [20]:
c = cell[2][2]
target_c = 28
scalfac_c = target_c/c
print(scalfac_c)
round(scalfac_c)

2.1347558772677067


2

In [21]:
Tmatrix = np.eye(3)
Tmatrix[0,0] = round(scalfac_a)
Tmatrix[1,1] = round(scalfac_b)
Tmatrix[2,2] = round(scalfac_c)
Tmatrix

array([[21.,  0.,  0.],
       [ 0., 21.,  0.],
       [ 0.,  0.,  2.]])

In [22]:
supercell = ase.build.make_supercell(combined, Tmatrix)

In [23]:
v = view(supercell, viewer='ngl')
v

In [24]:
lammpsdata.write_lammps_data("Al2O3_supercell.data", supercell)

In [25]:
second_slab = supercell.copy()

In [26]:
trans = np.zeros_like(supercell.positions)
trans[:,2] += (20 + supercell.cell[2][2])

In [27]:
second_slab.set_positions(second_slab.positions + trans)

In [28]:
combined_slabs = supercell + second_slab

In [29]:
combined_slabs.cell[2][2] += (20 +supercell.cell[2][2])

In [30]:
v = view(combined_slabs, viewer='ngl')
v

In [50]:
lammpsdata.write_lammps_data("combined_cells.data", combined_slabs)

In [32]:
from ase.build import bulk

In [43]:
NiBulk = bulk('Ni', 'fcc', a=3.528, orthorhombic=True)
NiBulk.cell

Cell([2.4946727240261395, 2.4946727240261395, 3.528])

In [37]:
v = view(NiBulk, viewer='ngl')
v

In [46]:
nisalb = ase.build.fcc100("Ni", (40,40, 10), a=3.528, vacuum=None, orthogonal=True, periodic=True)

In [47]:
nisalb.cell

Cell([99.7869089610456, 99.7869089610456, 17.64])

In [48]:
101/2.467272402614

40.93589337480271

In [49]:
v = view(nisalb, viewer='ngl')
v

In [52]:
combined = ase.build.stack(combined_slabs, nisalb)

IncompatibleCellError: *maxstrain* exceeded. *atoms1* strained 0.791025 and *atoms2* strained 0.791025.